In [9]:
import os
import numpy as np
from nilearn import image, plotting
import nibabel as nb
from functions import edit_events_musicnoise, extract_samples, convert_samples_to_features, edit_events_full, edit_events_factors

In [10]:
data_root = '/Volumes/T7/BIDS-BRAINPLAYBACK-TASK2'
fmriprep_dir = os.path.join(data_root, 'derivatives','fmriprep23')
func_dir = os.path.join(data_root, 'derivatives', 'mvpa_04_musicnoise_bold', 'func_clean')
stab_mask_dir = os.path.join(data_root, 'derivatives', 'mvpa_09_stabilityperfold_bold', 'stab_full')
#output_feat_musicnoise_dir = os.path.join(data_root, 'derivatives', 'mvpa_08_stability_musicnoise_bold', 'features_musicnoise_stab_full')
#output_feat_factors_dir = os.path.join(data_root, 'derivatives', 'mvpa_08_stability_musicnoise_bold', 'features_factors_stab_full')
output_feat_full_dir = os.path.join(data_root, 'derivatives', 'mvpa_09_stabilityperfold_bold', 'features_stab_full')

In [11]:
subject = '02'

In [12]:
n_folds = 4
if subject == '18': n_folds = 3

example_func = nb.load(os.path.join(func_dir,f'sub-{subject}_ses-01_task-02a_run-1_cleaned.nii.gz'))

for ff in range(n_folds):

    print('--'*10)
    print(f'Fold {ff}...')

    stab_mask_data = np.load(os.path.join(stab_mask_dir, f'sub-{subject}_STAB_fold_{ff}.npy'))

    # generate image
    stab_mask_img = nb.Nifti1Image(stab_mask_data, example_func.affine, example_func.header)

    # threshold the stability map
    thres = 0.15
    stab_mask_img_thres = image.math_img("img > thres", img=stab_mask_img, thres=thres)
    #stab_mask_img_thres = image.threshold_img(stab_mask_img, threshold=thres)
    stab_mask_img_thres_bin = image.binarize_img(stab_mask_img_thres)

    runList = ['1','2','3','4']
    if subject == '18': runList = ['1','3','4']

    for run in runList:    
        img_clean = nb.load(os.path.join(func_dir, f'sub-{subject}_ses-01_task-02a_run-{run}_cleaned.nii.gz'))
        samples = extract_samples(img_clean, stab_mask_img_thres_bin, subject, run)

        events_split_2 = edit_events_full(data_root, subject, run)
        convert_samples_to_features(samples, output_feat_full_dir, events_split_2, subject, run, ff)


--------------------
Fold 0...


TypeError: ("Input images cannot be compared, you provided 'dict_values([<nibabel.nifti1.Nifti1Image object at 0x31e11ebd0>, 0.15])',", 'Data given cannot be loaded because it is not compatible with nibabel format:\n0.15')